# Neural Networks

Neural networks can be constructed using the `torch.nn` package.

Now that, you had a glimpse of `autograd`, `nn` depens on `autograd` to define models and differentiate them. An `nn.Module` contains layers, and method `forward(input)` that returns the `output`.

![](https://pytorch.org/tutorials/_images/mnist.png)

It is a simple feed-forward network. It takes the input, feeds it through several layers one after the other, and then finally gives the output.

A typical training procedure for a neural network is as follows:

* Define the neural network that has some learnable parameters(or weights)
* Iterate over a dataset inputs
* Processs input through the network
* Compute the loss (how far is the output from being correct)
* Propagate gradients back into the network's parameters
* Update the weights of the network, typically using  a simple update rule: `weight = weight - learning_rate * gradient`

## Define the network

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # an affine operation y = Wx + b
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimesion
        num_features = 1
        for s in size:
            num_features *= s
            
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


You just have to define the `forward` function, and the `backward` function (where gradients are computed) is automatically defined for you using `autograd`. You can use any of the Tensor operations in the `forward` function.

In [7]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [8]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0470,  0.0183,  0.1484,  0.2142,  0.0148, -0.1240, -0.0819, -0.0527,
         -0.0665,  0.0174]], grad_fn=<AddmmBackward>)


**Zero the gradient buffers of all parameters and backprops with random gradients:**

In [23]:
net.zero_grad()
out.backward(torch.randn(1, 10))

> **`torch.nn` only supports mini-batches. The entire `torch.nn` package only supports inputs that are a mini-batch of samples, and not a single sample.**

> **For exmaple, `nn.Conv2d` will takes in a 4D Tensor of `nSamples x nChannels x Height x Weight`**

> **If you have a single sample, just use the `input.unsqueeze(0)` to add a fake batch dimension.**

**Recap:**


* `torch.Tensor` - A multi-dimensional array with support for autograd like `backward()`. Also holds the gradient w.r.t the tensor.
* `nn.Module` - Neural network module. Convenient way of encapsulation parameters, with helpers for moving them to GPU exporting, loading, etc.
* `nn.Parameter` - A kind of Tensor, that is automatically registered as a parameter when assigned an attribute to a `Module`
* `autograd.Function` - Implements forward and backward definitions of an autograd operation. Every `Tensor` operation creates at least a single `Function` node that connects to functions that created a `Tensor` and encoes its history.

At this point, we covered:

* Defining a neural network
* Processing inputs and calling backward

Still left:

* Computing the loss
* Updating the weigths of the network

## Loss Function

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

These are several different loss functions under the nn package. A simple is `nn.MSELoss` which computes the mean-squared error between the input and the target.

For example:

In [15]:
import torch

In [16]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9929, grad_fn=<MseLossBackward>)


Now, if you follow `loss` in the backward direction, using its `.grad_fn` atrribute, you will see a graph of computattions that looks like this:

```
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
    -> view -> linear -> relu -> linear -> relu -> linear
    -> MSELoss
    -> loss
```

So, when you call `loss.backward()`, the whole graph is differentiated w.r.t the loss, and all Tensors in the graph that has `requires_grad=True` will have their `.grad` Tensor accumulated with the gradient.

In [12]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

## Backprop

**To backpropagate the error all we have to do is to `loss.backward()`. You need to clear the existing gradient though, else gradients will be accumulated to existing gradients.**

In [17]:
net.zero_grad()  # zeros the gradients buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0163,  0.0178,  0.0072,  0.0073, -0.0068,  0.0017])


## Update the weights

The simplest update rule used in practice is the Stochastic Gradient Descent(SGD):

`weight = weight - learning_rate * gradient`

In [25]:
1e-2, 1e2

(0.01, 100.0)

In [26]:
learning_rate = 1e-2
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

However,as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: `torch.optim` that implements all these methods. Using it is very simple.

In [28]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=1e-2)

# in your training loop:
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()  # Does the update

> Observe how gradient buffers had to be manually set to zero using `optimizer.zero_grad()`. This is because gradients are accumulated as explained in the Backprop section.